In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import seaborn as sns
db = sns.load_dataset('tips')

In [8]:
db.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [20]:
db.shape

(244, 7)

In [26]:
db.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [30]:
db['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [39]:
db['sex']=np.where(db['sex']=='Male',1,0)

In [43]:
db['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [45]:
db['smoker']=np.where(db['smoker']=='Yes',1,0)

In [64]:
db['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [66]:
db['time']=np.where(db['time']=='Dinner',0,1)

In [68]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    int64   
 3   smoker      244 non-null    int64   
 4   day         244 non-null    category
 5   time        244 non-null    int64   
 6   size        244 non-null    int64   
dtypes: category(1), float64(2), int64(4)
memory usage: 12.0 KB


In [75]:
db.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [77]:
x = db[[ 'tip', 'sex', 'smoker', 'day', 'time', 'size' ]]
y = db['total_bill']

In [79]:
x.head()

,tip,sex,smoker,day,time,size
0,1.01,0,0,Sun,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
3,3.31,1,0,Sun,0,2
4,3.61,0,0,Sun,0,4


In [81]:
from sklearn.model_selection import train_test_split

In [83]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=10)

In [85]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [89]:
ct = ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')

In [97]:
import sys
np.set_printoptions(threshold=sys.maxsize)
x_train = ct.fit_transform(x_train)

In [101]:
x_test = ct.transform(x_test)

In [133]:
from sklearn.svm import SVR
svr = SVR()

In [135]:
svr.fit(x_train,y_train)

SVR()

In [137]:
y_pred = svr.predict(x_test)

In [139]:
from sklearn.metrics import r2_score
pram = {
    'C':[0.1,1,10,100,1000],
    'gamma':[1,0.1,0.01,0.001,0.0001],
    'kernel':['rbf']
}

In [141]:
print(r2_score(y_test,y_pred))

0.4602811456115927


In [143]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVR(),pram,refit=True,verbose=3)

In [145]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [147]:
y_pred = grid.predict(x_test)

In [149]:
print(r2_score(y_test,y_pred))

0.5081618245078687


In [151]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}